In [ ]:
class Point:
  def __init__(self, x, y):
    self.x = x
    self.y = y

def printPointList(points):
  print()
  for p in points:
    printPoint(p)

def printPoint(point):
  print("(", point.x, ", ", point.y, ")", sep="", end=" ")

def receiveInput():
    inputPoints = input().split()

    points = []

    for i in range(0, len(inputPoints) - 1, 2):
      points.append(Point(int(inputPoints[i]), int(inputPoints[i + 1])))

# points = receiveInput()

points = [(0, 3), (1, 2), (4, 4), (0, 0), (3, 1), (3, 3), (4, 2), (-2, 1)]

points = [Point(x, y) for x, y in points]